<a href="https://colab.research.google.com/github/Mal-web-cmd/Python_Libraries/blob/main/Week_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pytube moviepy opencv-python


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 2.4 MB/s eta 0:00:00


In [2]:
!pip install --upgrade pytube

In [3]:
!pip install yt-dlp


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.3/173.3 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 41.8 MB/s eta 0:00:00


In [4]:
!apt-get update
!apt-get install ffmpeg


Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,720 kB]
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,927 kB]
Get:13 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1

In [5]:
!ffmpeg -version


ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enable-l

In [13]:
import os
print("Current working directory:", os.getcwd())


Current working directory: /content


In [14]:
import os
for f in os.listdir():
    print(f)


.config
audio.wav
video.mp4
sample_data


In [15]:
import os
print(os.path.exists('video.mp4'))  # This will return True if the file exists


True


In [9]:
import subprocess
import yt_dlp as youtube_dl
import moviepy.editor as mp
import cv2
import numpy as np

# Download the YouTube video using yt-dlp
def download_video(url):
    try: # Corrected indentation here
        print(f"Attempting to download video from URL: {url}")

        # Force both video and audio to be merged into a single mp4 named video.mp4
        ydl_opts = {
            'format': 'bestvideo+bestaudio/best',
            'outtmpl': 'video.%(ext)s',
            'merge_output_format': 'mp4',
        }

        with youtube_dl.YoutubeDL(ydl_opts) as ydl:
            info_dict = ydl.extract_info(url, download=True)
            video_path = 'video.mp4' # We now know this will be the output
            print(f"Video downloaded successfully: {video_path}")
            return video_path
    except Exception as e: # Corrected indentation here
        print(f"Error downloading video: {e}")
        return None



# Convert video to MP4 (if needed)
def convert_video_to_mp4(input_path, output_path='video_converted.mp4'):
    try: # Corrected indentation here
        # Use FFmpeg to convert the video
        subprocess.run(['ffmpeg', '-i', input_path, '-c:v', 'libx264', '-c:a', 'aac', '-strict', 'experimental', output_path])
        print(f"Video converted to MP4 successfully: {output_path}")
        return output_path
    except Exception as e: # Corrected indentation here
        print(f"Error converting video: {e}")
        return None

# Extract audio and frames
def extract_audio_and_frames(video_path):
    try: # Corrected indentation here
        # Convert video to MP4 if it's not already in a format MoviePy can handle
        if not video_path.endswith('.mp4'):
            video_path = convert_video_to_mp4(video_path)

        # Load the video using moviepy
        video = mp.VideoFileClip(video_path)

        # Extract audio and save it as a .wav file
        audio_path = "audio.wav"
        video.audio.write_audiofile(audio_path)

        # Extract frames using OpenCV
        cap = cv2.VideoCapture(video_path)
        frames = []
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break
            frames.append(frame)
        cap.release()

        return audio_path, frames
    except Exception as e:  # Corrected indentation here
        print(f"Error extracting audio or frames: {e}")
        return None, None

# Main function to process the given video URL
def process_video(url):
    # Download the video
    video_path = download_video(url)
    if video_path is None: # Exit if download failed
        return None, None

    # Extract audio and frames
    audio_path, frames = extract_audio_and_frames(video_path)

    if audio_path and frames:
        print(f"Audio extracted to: {audio_path}")
        print(f"Extracted {len(frames)} frames from the video.")

    return audio_path, frames

# Test with the given YouTube video URL
url = "https://www.youtube.com/watch?v=7BOCJFBxiXQ"
audio_path, frames = process_video(url) # Call the function with the URL to get the results

  if event.key is 'enter':



Attempting to download video from URL: https://www.youtube.com/watch?v=7BOCJFBxiXQ
[youtube] Extracting URL: https://www.youtube.com/watch?v=7BOCJFBxiXQ
[youtube] 7BOCJFBxiXQ: Downloading webpage
[youtube] 7BOCJFBxiXQ: Downloading tv client config
[youtube] 7BOCJFBxiXQ: Downloading player ab11fe69-main
[youtube] 7BOCJFBxiXQ: Downloading tv player API JSON
[youtube] 7BOCJFBxiXQ: Downloading ios player API JSON
[youtube] 7BOCJFBxiXQ: Downloading m3u8 information
[info] 7BOCJFBxiXQ: Downloading 1 format(s): 401+251
[download] Destination: video.f401.mp4
[download] 100% of  367.55MiB in 00:00:06 at 53.35MiB/s  
[download] Destination: video.f251.webm
[download] 100% of   11.99MiB in 00:00:00 at 48.51MiB/s  
[Merger] Merging formats into "video.mp4"
Deleting original file video.f401.mp4 (pass -k to keep)
Deleting original file video.f251.webm (pass -k to keep)
Video downloaded successfully: video.mp4
MoviePy - Writing audio in audio.wav


MoviePy - Done.


In [10]:
!pip install -U openai-whisper
!sudo apt-get install ffmpeg


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.5/800.5 kB 11.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 46.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 40.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 54.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 36.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s e

In [11]:
import cv2
import subprocess
import whisper # Added import statement for whisper

# Speech recognition

# Load once
whisper_model = whisper.load_model("base")  # Try "small" or "medium" for better accuracy

def convert_audio_to_whisper_format(input_path, output_path="converted_audio.wav"):
    command = [
        'ffmpeg',
        '-y',  # Overwrite output file if exists
        '-i', input_path,
        '-ar', '16000',        # Sample rate
        '-ac', '1',            # Mono channel
        '-c:a', 'pcm_s16le',   # 16-bit PCM
        output_path
    ]
    subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    return output_path
def transcribe_audio_whisper(audio_path):
    print("[INFO] Transcribing using Whisper...")

    # 🔁 Step 1: Convert audio to whisper-friendly format
    converted_path = convert_audio_to_whisper_format(audio_path)

    try:
        # 🔁 Step 2: Use Whisper to transcribe
        result = whisper_model.transcribe(converted_path)
        print("[INFO] Whisper Transcription Completed.")
        return result["text"]
    except Exception as e:
        print(f"[ERROR] Whisper transcription failed: {e}")
        return ""
# Scene change detection using frame differences
def detect_scene_changes(frames, threshold=30):
    scene_changes = []
    previous_frame = None
    for i, frame in enumerate(frames):
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        if previous_frame is not None:
            diff = cv2.absdiff(previous_frame, gray_frame)
            non_zero_count = np.count_nonzero(diff)
            if non_zero_count > threshold:
                scene_changes.append(i)
        previous_frame = gray_frame
    return scene_changes

100%|████████████████████████████████████████| 139M/139M [00:01<00:00, 139MiB/s]


In [12]:

# Classify key moments based on transcribed speech
def classify_key_moments(transcription):
    key_moments = {
        "unboxing": ["unbox", "opening", "packaging"],
        "feature_demo": ["feature", "demo", "demonstrate"],
        "final_verdict": ["verdict", "conclusion", "final thoughts"]
    }

    moments = []

    if not transcription or not isinstance(transcription, str):
        print("[WARN] Transcription is empty or invalid.")
        return moments  # Return empty list instead of crashing

    for moment, keywords in key_moments.items():
        if any(keyword in transcription.lower() for keyword in keywords):
            moments.append(moment)

    return moments
#Segment the video into chunks based on scene changes
def segment_video_by_scene_changes(video_path, frames, scene_changes, segment_duration=30):
    segments = []
    for i in range(0, len(scene_changes), segment_duration):
        start_frame = scene_changes[i]
        end_frame = scene_changes[min(i + segment_duration, len(scene_changes) - 1)]

        # Calculate the start and end time for the segment based on frame indices
        start_time = start_frame / float(len(frames)) * mp.VideoFileClip(video_path).duration
        end_time = end_frame / float(len(frames)) * mp.VideoFileClip(video_path).duration

        # Save each segment as a new video file
        segment_file = f"segment_{i // segment_duration + 1}.mp4"
        video_clip = mp.VideoFileClip(video_path).subclip(start_time, end_time)
        video_clip.write_videofile(segment_file, codec="libx264")

        segments.append(segment_file)
        video_clip.close()  # Close the video clip to release resources

    return segments
